<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/Mnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LogisticRegression

In [ ]:
file_path = Path("HPQ.csv")
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df=df[df.FutureTrend !='Neutral']
df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,29.180000,29.510000,28.959999,29.200001,7914000,0.436366,0.068542,Negative,Positive,Positive,0,Negative
750,24/06/2021,29.500000,29.510000,29.110001,29.469999,6270700,0.899998,-0.101697,Negative,Negative,Negative,0,Negative
751,25/06/2021,29.400000,29.920000,29.370001,29.709999,17050100,0.618179,1.054420,Positive,Negative,Positive,0,Negative
752,28/06/2021,29.760000,30.030001,29.639999,29.809999,6570000,0.435896,0.168008,Negative,Positive,Positive,0,Negative
753,29/06/2021,29.950001,30.320000,29.790001,30.139999,7779100,0.660376,0.634386,Negative,Negative,Positive,0,Negative


In [ ]:
# Setting Index as Date
appl_df = df.set_index("Date")
appl_df.tail()

,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
Date,,,,,,,,,,,,
23/06/2021,29.180000,29.510000,28.959999,29.200001,7914000,0.436366,0.068542,Negative,Positive,Positive,0,Negative
24/06/2021,29.500000,29.510000,29.110001,29.469999,6270700,0.899998,-0.101697,Negative,Negative,Negative,0,Negative
25/06/2021,29.400000,29.920000,29.370001,29.709999,17050100,0.618179,1.054420,Positive,Negative,Positive,0,Negative
28/06/2021,29.760000,30.030001,29.639999,29.809999,6570000,0.435896,0.168008,Negative,Positive,Positive,0,Negative
29/06/2021,29.950001,30.320000,29.790001,30.139999,7779100,0.660376,0.634386,Negative,Negative,Positive,0,Negative


In [ ]:
target=df['FutureTrend']
df=df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
df['ema5'] = df['Close'] / df['Close'].ewm(5).mean()
# df['normVol'] = df['Volume'] / df['Volume'].ewm(5).mean()   
df['diff']=df['Open']-df['Close']
del (df['Open'])
del (df['Close'])

df.head()

,Date,High,Low,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,ema5,diff
0,02/07/2018,22.860001,22.379999,5855900,1.000000,1.419697,Positive,Negative,Positive,0,1.000000,-0.320000
1,03/07/2018,22.930000,22.549999,3619200,0.105265,-1.439790,Neutral,Neutral,Negative,0,0.994597,0.330000
2,05/07/2018,22.990000,22.450001,7158600,0.555555,-0.306747,Negative,Positive,Negative,0,1.000991,0.070000
3,06/07/2018,23.219999,22.600000,5375100,0.661291,1.009656,Negative,Negative,Positive,0,1.008396,-0.230000
4,09/07/2018,23.440001,23.010000,7057100,0.837210,0.993956,Negative,Positive,Positive,0,1.017320,-0.230001


In [ ]:
new_df=appl_df
new_df= df.set_index("Date")

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=4)
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.fit_transform(X_test)
from sklearn.decomposition import PCA
pca = PCA(n_components=7)
# prepare transform on dataset
pca.fit(X_train)
# apply transform to dataset
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
model =  LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C=1.0)
model.fit(X_train,y_train)

['Oil FutureTrend', 'Gold FutureTrend', 'Trend']


LogisticRegression(multi_class='multinomial')

In [ ]:
print("Accuracy score (training): {0:.3f}".format(
        model.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        model.score(
            X_test, 
            y_test)))
  

Accuracy score (training): 0.595
Accuracy score (validation): 0.510
